In [1]:
import pandas as pd
from bert_score import BERTScorer
from scipy.stats import zscore

from src.config import substitution_rating_file, substitution_rating_scores

In [2]:
import time
start = time.time()
scorer = BERTScorer(lang="en", rescale_with_baseline=True)
end = time.time()
print(end - start)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.dense.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


14.784382820129395


In [3]:
def get_scores(single_cands, multi_refs):
    P_mul, R_mul, F_mul = scorer.score([single_cands], [multi_refs])
    return P_mul.item(), R_mul.item(), F_mul.item()

def scores(line):
    return get_scores(line['item_2'].lower(), line['item_1'].lower())

In [59]:
# testing BERTScorer
X = ["Almond bars"]
Y = ["Cheese Burger", "Roasted Walnuts", "Hot coffee", "Jupiter and Zeus"]
print(get_scores(X[0].lower(), Y[0].lower()))
print(get_scores(X[0].lower(), Y[1].lower()))
print(get_scores(X[0].lower(), Y[2].lower()))
print(get_scores(X[0].lower(), Y[3].lower()))

(0.30472931265830994, 0.2820392847061157, 0.2945074141025543)
(0.34518226981163025, 0.21366684138774872, 0.279764324426651)
(0.21180827915668488, 0.4166193902492523, 0.3133380115032196)
(0.16183911263942719, 0.0072509958408772945, 0.08484045416116714)


In [4]:
df = pd.read_csv(substitution_rating_file)

In [5]:
import time
start = time.time()
df['P'], df['R'], df['F1'] = zip(*df.apply(scores, axis=1))
end = time.time()
print(end - start)

737.2809174060822


In [6]:
cols = ['user', 'item_1', 'item_2', 'rating', 'P', 'R', 'F1']
df = df[cols]
# df = df[cols].rename(columns={'R':'BERTScore', 'F1':'BERTScore-F1'})

In [7]:
filename = substitution_rating_scores['BERTScores']
df.to_csv(filename, index=False)

# Normalization

In [9]:
cols = ['rating_z', 'R', 'F1']

dfs = []
for u, df_temp in df.groupby('user'):
    df_temp['rating_z'] = zscore(df_temp['rating'])
    dfs.append(df_temp)
d1_z = pd.concat(dfs)
r2 = d1_z[cols].corr().head(1)
print(' & '.join([str(s) for s in r2.round(3).values[0][1:]]), '\n')
r2

0.203 & 0.246 



,rating_z,R,F1
rating_z,1.0,0.203448,0.245858
